# 0. Установка репозитория и библиотек (~5мин)

In [ ]:
!git clone https://github.com/ernygati/NeuroVoice.git
import os
os.chdir("/kaggle/working/NeuroVoice")
!pip install pyyaml>=6.0.2 transformers==4.35.2 tts==0.22.0 torch==2.5.1 pyzmq==26.4.0 ipykernel==6.20.2 soxr==0.3.7 graphviz==0.20.3 matplotlib==3.8.0 onnx==1.16.2 onnxruntime==1.16.3 openpyxl==3.1.2 tensorboard==2.19.0 tensorboard-data-server==0.7.2 torchmetrics==1.7.0 torchview==0.2.6 tqdm==4.67.1 typing-extensions==4.13.0 openai-whisper>=20250625 -q

Cloning into 'NeuroVoice'...
remote: Enumerating objects: 304, done.
remote: Counting objects: 100% (304/304), done.
remote: Compressing objects: 100% (203/203), done.
remote: Total 304 (delta 60), reused 290 (delta 49), pack-reused 0 (from 0)
Receiving objects: 100% (304/304), 5.72 MiB | 16.98 MiB/s, done.
Resolving deltas: 100% (60/60), done.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
dask-expr 1.1.21 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which 

# 1. Проверка готовности файлов для обучения

В случае, если необходимые файлы будут отсуствовать, запустится код по созданию этих файлов (⚠️ НУЖНО GPU ⚠️)

Советы:
- для старта обучения с предобученной модели укажите путь до .pth файла, задав переменную `CHECKPOINT_PATH`. Если вы начинаете обучение с 0, то оставьте это поле `None`

In [10]:
import shutil
import yaml
from pipelines.train import TrainPipeline
from utils.create_config import create_new_config

#путь до главного конфига с указанием путей до основных папок  файлов
CONFIG_PATH="/kaggle/working/NeuroVoice/configs/train/shtefanov.yaml" #!НАСТРОЙТЕ
#путь до готовых файлов обучения (если имеется)
TRAIN_FILES = "/kaggle/input/shtefanov-bd/shtefanov_pope"             #!НАСТРОЙТЕ
CHECKPOINT_PATH= None                                                 #!НАСТРОЙТЕ


config = yaml.safe_load(open(CONFIG_PATH))

if not os.path.exists(CONFIG_PATH):
    create_new_config()
config["data"]["base_wav_path"] = "/kaggle/input/shtefanov-bd/shtefanov_pope.wav"
config["models"]["whisper"] = "large-v3"
config["data"]["checkpoint_path"]=CHECKPOINT_PATH

if os.path.exists(TRAIN_FILES) and "wavs" in os.listdir(TRAIN_FILES):
    voice_folderpath=os.path.join(config["data"]["main_dir"], os.path.basename(TRAIN_FILES))
    if os.path.exists(voice_folderpath):
        shutil.rmtree(voice_folderpath)
    shutil.copytree(TRAIN_FILES, voice_folderpath)
trainpipe=TrainPipeline(config)
trainpipe._preprocess_data()

😊 All mandatory files are ready!
😊 wavs/ folder is not empty!
😊 All mandatory files are ready!
😊 wavs/ folder is not empty!


('data/shtefanov/xtts/shtefanov_pope/metadata_train.csv',
 'data/shtefanov/xtts/shtefanov_pope/metadata_eval.csv')

# 3. Запуск обучения модели (⚠️ НУЖНО GPU ⚠️)

Обученные веса вы можете найти в папке, указанной в конфиге: `config["data"]["weights_folder"]`

In [11]:
# запустить обучение модели
trainpipe.train()

😊 All mandatory files are ready!
😊 wavs/ folder is not empty!
😊 All mandatory files are ready!
😊 wavs/ folder is not empty!


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

LICENSE.txt: 0.00B [00:00, ?B/s]

...

zh-cn-sample.wav:   0%|          | 0.00/383k [00:00<?, ?B/s]

speakers_xtts.pth:   0%|          | 0.00/7.75M [00:00<?, ?B/s]

pt_sample.wav:   0%|          | 0.00/445k [00:00<?, ?B/s]

Starting from existing checkpoint /kaggle/input/shtefanov-bd/best_model_35.pth ...


KeyboardInterrupt: 

In [12]:
config

{'name': 'shtefanov',
 'device': 'cuda:0',
 'audio': {'min_duration': 2.0, 'max_duration': 10.0},
 'data': {'main_dir': 'data/shtefanov/xtts',
  'base_wav_path': '/kaggle/input/shtefanov-bd/shtefanov_pope.wav',
  'weights_folder': 'out/shtefanov/xtts',
  'checkpoint_path': '/kaggle/input/shtefanov-bd/best_model_35.pth',
  'reference_audio': '<PASTE_PATH>'},
 'models': {'whisper': 'large-v3', 'VAD_folder': 'models/VAD'},
 'train': {'epochs': 15, 'batch_size': 2}}

# (Опционально): Запустить скачивание .wav файла для обучения с Youtube


In [ ]:
from pytube import YouTube
import librosa
import os

def download_youtube_wav_pytube(url, output_path="audio.wav"):
    """Download YouTube audio using pytube"""
    yt = YouTube(url)
    
    # Get audio stream
    audio_stream = yt.streams.filter(only_audio=True).first()
    
    # Download as MP4
    temp_file = audio_stream.download(filename="temp_audio")
    
    # Convert to WAV using librosa (or you can use ffmpeg)
    audio, sr = librosa.load(temp_file, sr=22050)
    librosa.output.write_wav(output_path, audio, sr)
    
    # Clean up
    os.remove(temp_file)
    
    return output_path

# Usage
url = "https://youtu.be/EqLevnBwMFA?si=JIPfdB8G-2ZI0WWq"
wav_file = download_youtube_wav_pytube(url)